In [9]:
import json
import pandas as pd
import plotly.express as px
import re
import numpy as np

In [ ]:
# Load the coordinates from the JSON file
with open("../../Extract/coordGPS/coordinates.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# Convert the raw JSON data into a pandas DataFrame
# Latitude and longitude values are explicitly converted to floats to ensure
# proper numeric types for calculations, mapping, or geospatial analysis.
df_coords = pd.DataFrame([
    {"city": city, "latitude": float(data["latitude"]), "longitude": float(data["longitude"])}
    for city, data in raw_data.items()
])

df_coords.head()


,city,latitude,longitude
0,"Besançon, France",47.238022,6.024362
1,"Bormes-les-Mimosas, France",43.150697,6.341928
2,"Nimes, France",43.837425,4.360069
3,"Dijon, France",47.321581,5.041470
4,"Lille, France",50.636565,3.063528


In [11]:
# Helpers

def clean_city_name(name: str) -> str:
    """
    Clean a city name for display:
    - Removes trailing ", France"
    - Collapses extra spaces
    - Keeps capitalization
    """
    name = re.sub(r",?\s*France$", "", name, flags=re.I)
    name = re.sub(r"\s+", " ", name).strip()
    return name


# Cleaning 

# Create a clean label for display (remove ", France" and extra spaces)
df_coords["city_clean"] = df_coords["city"].apply(clean_city_name)

# Label only for important cities (display purpose)
important_cities = {
    "Paris", "Lyon", "Marseille", "Toulouse", "Lille",
    "Strasbourg", "Grenoble", "Dijon", "Nîmes",
    "Avignon", "La Rochelle", "Biarritz",
}

df_coords["label"] = np.where(
    df_coords["city_clean"].isin(important_cities),
    df_coords["city_clean"],
    ""
)


# Plot 
fig = px.scatter_mapbox(
    df_coords,
    lat="latitude",
    lon="longitude",
    hover_name="city_clean",
    height=720,
    zoom=5.2,
)

# Add larger markers and display labels above each point
fig.update_traces(
    mode="markers+text",
    marker=dict(color="red", size=9),
    text=df_coords["label"],
    textposition="top center",
    textfont=dict(size=11),
)

# Layout
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_center={"lat": 46.6, "lon": 2.0},
    title="Carte des 35 villes touristiques françaises — Coordonnées GPS",
    title_x=0.5,
    margin={"r": 0, "t": 50, "l": 0, "b": 0},
)

fig.show()
